In [1]:
import pandas as pd
import time
import warnings
from apriori_algorithm import AprioriAlgorithm
from preprocessor import Streamer, preprocess

warnings.filterwarnings('ignore')

In [2]:
dataset_metadata = {
    'amazon-reviews': {
        'path': './data/amazon-reviews/all_csv_files.csv',
        'size': 233055326//2,  # original size = 233055326
        'limit': 10000
    },
    
    'groceries': {
        'path': './data/groceries/Groceries_dataset.csv',
        'size': 38766,
        'limit': 10000
    },
    
    
    'movielens': {
        'path': ['./data/movielens/ratings.csv', './data/movielens/movies.csv'],
        'size': 100836, 
        'limit': 50000
    }
}

In [3]:
MIN_SUPPORT = 100
VERBOSE = False
DISPLAY_ITERATION = 5000

In [4]:
apriori = AprioriAlgorithm(minsup=MIN_SUPPORT, verbose=VERBOSE)

In [5]:
datasets = [
    'movielens',
    'groceries',
    'amazon-reviews',
    ]

In [6]:
threshold_rating = 4.0

In [7]:
for dataset in datasets:
    
    results = pd.DataFrame(columns=['item_sets', 'supp_count'])
    dataset_meta = dataset_metadata[dataset]
    num_streams = dataset_meta['size'] // dataset_meta['limit']
    
    if dataset == 'movielens':
        datastream = Streamer(filepath=dataset_meta['path'][0],
                              stream_limit=dataset_meta['limit'])
        movies_df = pd.read_csv(dataset_meta['path'][1])

    else:
        datastream = Streamer(filepath=dataset_meta['path'],
                            stream_limit=dataset_meta['limit'])
    
    print(f'Running APRIORI on {dataset} dataset with {num_streams} sequential I/O streams')
    print("##"*30)
    
    for stream_id in range(num_streams):

        start_time = time.time()
        
        df = datastream.getCurrentStream(stream_id)

        if dataset == 'movielens':
            movie_transactions = preprocess(dataset_name=dataset,
                                            ratings=df,
                                            movies=movies_df,
                                            threshold_rating=threshold_rating)

        else:
            movie_transactions = preprocess(dataset_name=dataset, 
                                            df=df, 
                                            threshold_rating=threshold_rating)
        print(f'Number of transactions: {len(movie_transactions)}')
            
    
        freq_item_sets = apriori.run(movie_transactions)
        print(f'Number of frequent itemsets found: {len(freq_item_sets)}')
        
        results = pd.concat([results, freq_item_sets], ignore_index=True)
        results = results.drop_duplicates()
        
            
        time_taken = round(time.time() - start_time, 2)

        print(f'Finished data-stream #{stream_id} in {time_taken} seconds.')
        # display(results) # uncomment to display results after each stream
                    
        print('--'*30)
    
    print(f'Completed {dataset} dataset.')
    display(results)
    results.to_csv(f'results_{dataset}_minsup_{MIN_SUPPORT}.csv', index=False)
    
    print("##"*30, '\n')

Running APRIORI on movielens dataset with 2 sequential I/O streams
############################################################
Accessing data-stream #0
Number of transactions: 322
Number of frequent itemsets found: 8
Finished data-stream #0 in 0.24 seconds.
------------------------------------------------------------
Accessing data-stream #1
Number of transactions: 288
Number of frequent itemsets found: 5
Finished data-stream #1 in 0.2 seconds.
------------------------------------------------------------
Completed movielens dataset.


,item_sets,supp_count
0,"( The (1991), The (1994))",105
1,"( The (1991), Silence of the Lambs)",114
2,"( The (1994), The (1995))",106
3,"( The (1994), The (1999))",119
4,"( The (1994), Forrest Gump (1994))",110
5,"( The (1994), Pulp Fiction (1994))",102
6,"( The (1994), Shawshank Redemption)",143
7,"( The (1999), Matrix)",119
8,"( The (1991), Silence of the Lambs)",111
9,"( The (1994), The (1999))",111


############################################################ 

Running APRIORI on groceries dataset with 3 sequential I/O streams
############################################################
Accessing data-stream #0
Number of transactions: 3512
Number of frequent itemsets found: 4
Finished data-stream #0 in 0.25 seconds.
------------------------------------------------------------
Accessing data-stream #1
Number of transactions: 3664
Number of frequent itemsets found: 1
Finished data-stream #1 in 0.33 seconds.
------------------------------------------------------------
Accessing data-stream #2
Number of transactions: 3567
Number of frequent itemsets found: 27
Finished data-stream #2 in 0.34 seconds.
------------------------------------------------------------
Completed groceries dataset.


,item_sets,supp_count
0,"(frankfurter, sausage)",109
1,"(frankfurter, whole milk)",102
2,"(sausage, tropical fruit)",108
3,"(sausage, whole milk)",146
4,"(other vegetables, whole milk)",121
5,UHT-milk,100
6,bottled beer,159
7,bottled water,261
8,brown bread,194
9,butter,191


############################################################ 

Running APRIORI on amazon-reviews dataset with 11652 sequential I/O streams
############################################################
Accessing data-stream #0
Number of transactions: 5484
Number of frequent itemsets found: 14
Finished data-stream #0 in 0.15 seconds.
------------------------------------------------------------
Accessing data-stream #1
Number of transactions: 5149
Number of frequent itemsets found: 9
Finished data-stream #1 in 0.1 seconds.
------------------------------------------------------------
Accessing data-stream #2
Number of transactions: 7841
Number of frequent itemsets found: 3
Finished data-stream #2 in 0.08 seconds.
------------------------------------------------------------
Accessing data-stream #3
Number of transactions: 7952
Number of frequent itemsets found: 16
Finished data-stream #3 in 0.26 seconds.
------------------------------------------------------------
Accessing data-stream #4
Nu

TypeError: sequence item 0: expected str instance, list found